In [1]:
# prompt: !pip install torch, torchvision, dataset
!pip install torch torchvision torchaudio datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# Loading the required moduels
##  load_dataset class
##  AutoModelForSeq2SeqLM module
##  AutoTokenizer To tokenize things
## GenerationConfig to check the generation configuration








In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Load Dialogue sum dataset

In [4]:
huggingface_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

# Exploring the Dataset

In [5]:
example_indices = [40,200]
dash_line = '_'.join('' for i in range(100))
print(dash_line)

for idx, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', idx+1)
    print(dash_line)
    print('INPUT DIALOGUE: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

___________________________________________________________________________________________________
___________________________________________________________________________________________________
Example  1
___________________________________________________________________________________________________
INPUT DIALOGUE: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________

___

# Loading the model using AutoModelForSeq2SeqLM module

In [6]:
model_name = 'google/flan-T5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Tokenizing the data suing AutoTokenizer module of transformers

In [7]:
# dir(AutoModelForSeq2SeqLM)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# dir(AutoTokenizer)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Encoding the stence into embbedings using the tokenizer instance of AutoTokenizer

In [9]:
sentence = 'what time is it, Tome?'
sentence_encoded  = tokenizer(sentence, return_tensors='pt')
print(sentence_encoded['input_ids'][0])

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens=True)

print(sentence_decoded)

tensor([ 125,   97,   19,   34,    6, 3059,   15,   58,    1])
what time is it, Tome?


# Summarize dialogues with fan-t5 model witout prompt engineering

In [10]:
example_indices = [40,200]
dash_line = '_'.join('' for i in range(100))
print(dash_line)

for idx, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    inputs = tokenizer(dialogue, return_tensors='pt')
    print(inputs['input_ids'][0])
    model_output = model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    print(model_output.shape)
    output = tokenizer.decode(model_output
        ,
                              skip_special_tokens = True)
    print(dash_line)
    print('Example ', idx+1)
    print(dash_line)
    # print('INPUT DIALOGUE: ')
    print(f'INPUT PROMPT: \n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY: \n{summary}')
    # print(dataset['test'][index]['summary'])
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n{output}\n')
    print()

___________________________________________________________________________________________________
tensor([ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
            6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
            3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
         4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
           10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
           47,    78,  1480,     5,    27,   398,    36,   326,   230,     5,
         1713,   345, 13515,   357,  4663,    10,   363,    31,     7,     8,
        23601,    58,  1713,   345, 13515,   536,  4663,    10,    27,   398,
         3579,     8,  4169,    18,    17,  9288,    17,    63,  2412,     5,
         1713,   345, 13515,   357,  4663,    10,   148,    31,   162,  2500,
           13,    97,   780,     5,    37, 14421,  2478,    19,   182,   885,
            5,    94,   751,    31,    17,

# Summarize dialogues with fan-t5 model with zero-shot instruction prompt in-context learning

In [11]:
example_indices = [40,200]

def make_prompt(index):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    prompt = f"""

    {dialogue}

    what is going on?
    """
    return prompt, dialogue, summary

for idx, index, in enumerate(example_indices):

  prompt, dialogue, summary = make_prompt(index)
  inputs = tokenizer(prompt, return_tensors='pt')
  model_output = model.generate(inputs['input_ids'],
                                max_new_tokens=50)[0]
  output = tokenizer.decode(model_output,
                            skip_special_tokens=True)

  print(dash_line)
  print('Example ', idx+1)
  print(dash_line)
  # print('INPUT DIALOGUE: ')
  print(f'INPUT PROMPT: \n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY: \n{summary}')
  # print(dataset['test'][index]['summary'])
  print(dash_line)
  print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n{output}\n')
  print()



___________________________________________________________________________________________________
Example  1
___________________________________________________________________________________________________
INPUT PROMPT: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
MODEL GENERATION - WITHOUT PROMPT ENGINEERING: 
The train is about to leave.


_________________________

# In-context zero shot learning

In [12]:
example_indices = [40,200]

def make_prompt(index):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    prompt = f"""
    dialogue:
    {dialogue}

    what is going on?
    """
    return prompt, dialogue, summary

for idx, index, in enumerate(example_indices):

  prompt, dialogue, summary = make_prompt(index)
  inputs = tokenizer(prompt, return_tensors='pt')
  model_output = model.generate(inputs['input_ids'],
                                max_new_tokens=50)[0]
  output = tokenizer.decode(model_output,
                            skip_special_tokens=True)

  print(dash_line)
  print('Example ', idx+1)
  print(dash_line)
  # print('INPUT DIALOGUE: ')
  print(f'INPUT PROMPT: \n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY: \n{summary}')
  # print(dataset['test'][index]['summary'])
  print(dash_line)
  print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n{output}\n')
  print()

___________________________________________________________________________________________________
Example  1
___________________________________________________________________________________________________
INPUT PROMPT: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
MODEL GENERATION - WITHOUT PROMPT ENGINEERING: 
The train is about to leave Tom's place.


_____________

# In-context oneshot learning

In [32]:
example_indices_full = [40]
example_indices_to_summarize = 200
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''
  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    prompt += f"""

dialogue:

{dialogue}
what is going on?
{summary}
    """
  dialogue = dataset['test'][200]['dialogue']
  prompt += f"""

dialogue:
{dialogue}

what is going on?
    """
  return prompt

In [33]:

one_shot_prompt = make_prompt(example_indices_full, example_indices_to_summarize)
print(one_shot_prompt)



dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
what is going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
    
  
dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also

In [34]:

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
model_output = model.generate(inputs['input_ids'],
                              max_new_tokens=50)[0]
output = tokenizer.decode(model_output,
                          skip_special_tokens=True)

print(dash_line)
print('Example ', idx+1)
print(dash_line)
# print('INPUT DIALOGUE: ')
print(f'INPUT PROMPT: \n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}')
# print(dataset['test'][index]['summary'])
print(dash_line)
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n{output}\n')
print()

___________________________________________________________________________________________________
Example  2
___________________________________________________________________________________________________
INPUT PROMPT: 
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
______

# In-context few-shot *learning*

In [44]:
example_indices_full = [40,80,120, 300,500]
example_indices_to_summarize = 200
few_shot_prompt = make_prompt(example_indices_full, example_indices_to_summarize)
print(few_shot_prompt)



dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
what is going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
    

dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set

In [46]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens = 10)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 0.1)
generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 0.5)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 1.0)
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
model_output = model.generate(inputs['input_ids'],
                            generation_config=generation_config)[0]
output = tokenizer.decode(model_output,
                          skip_special_tokens=True)

print(dash_line)
print('Example ', idx+1)
print(dash_line)
# print('INPUT DIALOGUE: ')
print(f'INPUT PROMPT: \n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}')
# print(dataset['test'][index]['summary'])
print(dash_line)
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n{output}\n')
print()

___________________________________________________________________________________________________
Example  2
___________________________________________________________________________________________________
INPUT PROMPT: 
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
______